# A Guided Tour of LSST Processed Visit Images (calexp)
<br>Author(s): **David Shupe** ([@stargaser](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@stargaser))
<br>Maintainer(s): **Douglas Tucker** ([@douglasleetucker](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@douglasleetucker))
<br>Level: **Introductory**
<br>Last Verified to Run: **2021-09-11**
<br>Verified Stack Release: **w_2021_33**

### Learning Objectives:

This notebook investigates LSST stack processed visit images, conventionally referred to as `calexp` objects. The `calexp` is a Butler dataset type that corresponds to a single processed, calibrated, and characterized CCD and the metadata associated with that image (mask plane, psf, etc.). We will explore the content of the `calexp` object and then compare to coadd images.

After working through this tutorial you should be able to:
1. Investigate the content of a calexp (and to some extent, other Stack objects).
2. Follow some best practices when working with `calexp` objects.

### Logistics
This notebook is intended to be run at `lsst-lsp-stable.ncsa.illinois.edu` or `data.lsst.cloud` from a local git clone of the [StackClub](https://github.com/LSSTScienceCollaborations/StackClub) repo.

### Setup

In [ ]:
# Site, host, and stack version
! echo $EXTERNAL_INSTANCE_URL
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
# General imports
import os
from pprint import pprint
import matplotlib.pyplot as plt

In [ ]:
# Stack imports
import lsst.daf.butler   as dafButler        #load the Butler to access data 
import lsst.afw.display  as afwDisplay       #load lsst.afw.display to gain access to image visualization routines.  
afwDisplay.setDefaultBackend('matplotlib')   #set the default display backend to matplotlib

## Retrieving and inspecting images

We will be using the DESC DC2 Run2.2i data set (which comprises a simulated 300 sq deg patch of 5 years of Rubin LSST WFD data). Here, we create a Butler for the data set we wish to examine and grab the calexp image we are interested in exploring. More information on the Butler can be found in other Stack Club tutorials.

In the first part of the tutorial, we will examine the calibrated exposure (`calexp`) for a single `visit`.  In the second part of the tutorial we will examine the calibrated image for a coadd `patch`. In the following cell, we grab both of these data objects.

In [ ]:
# Location of the DC2 Gen3 repository on this site
URL = os.getenv('EXTERNAL_INSTANCE_URL')
if URL.endswith('data.lsst.cloud'): # IDF
    repo = "s3://butler-us-central1-dp01"
elif URL.endswith('ncsa.illinois.edu'): # NCSA
    repo = "/repo/dc2"
else:
    raise Exception(f"Unrecognized URL: {URL}")

collection='2.2i/runs/DP0.1'

# Create the butler
butler = dafButler.Butler(repo,collections=collection)
registry = butler.registry        

# Grab a calexp of interest
dataId = {'visit': '512055', 'detector': 76, 'band':'i'}
calexp = butler.get('calexp', **dataId)

## Grab a deepCoadd tract and patch
coadd_dataId = {'band':'i', 'skymap': 'DC2', 'tract': 4851, 'patch': 29}
coadd = butler.get('deepCoadd', **coadd_dataId)    

## Image planes / pixel data

In terms of pixel data, a calexp contains an image, a mask, and a variance.

Let's see how to access the image.

In [ ]:
calexp.image

The `calexp.image` is a Stack object of type `ImageF` (an image with floating point values).

The `calexp.image` object can be displayed directly using other tools in the LSST Stack. To show the pixel data, we will make use of the matplotlib backend to `lsst.afw.display`.

Due to current limitations of this backend, the display must be defined and used in the same code cell, much as matplotlib commands in a notebook must all be in one cell to produce a plot.

In [ ]:
%matplotlib inline

If the entire calexp is displayed, masks will be overlaid. Here we will eschew the mask display by showing only the image.

In [ ]:
display = afwDisplay.Display()
display.scale("asinh", "zscale")
display.mtv(calexp.image)

We can also access the low-level pixel values as a `numpy` array, using the `.array` attribute.

In [ ]:
data = calexp.image.array
data

In [ ]:
data.__class__

Let's list all the methods for our calexp.

In [ ]:
calexp_methods = [m for m in dir(calexp) if not m.startswith('_')]

In [ ]:
calexp_methods

The `calexp` also contains a higher level `MaskedImage` object, which combines both the pixel values and the integer mask assigned to them.

In [ ]:
calexp.maskedImage

The `MaskedImage` can also be plotted directly by lsst.afw.display. More information can be found in the StackClub notebook on afw.display.

Access the variance object and the underlying Numpy array

In [ ]:
calexp.variance

In [ ]:
calexp.variance.array

Access the mask and its underlying array

In [ ]:
calexp.mask

In [ ]:
calexp.mask.array

Get the dimensions of the image, mask and variance

In [ ]:
calexp.getDimensions()

The image, maskedImage and Exposure objects in `lsst.afw.display` include information on **LSST pixels**, which are 0-based with an optional offset.

For a calexp these are usually zero.

In [ ]:
calexp.getXY0()

In [ ]:
calexp.getX0(), calexp.getY0()

## Metadata

Access the wcs object

In [ ]:
wcs = calexp.getWcs()
wcs

The WCS object can be used e.g. to convert pixel coordinates into sky coordinates

In [ ]:
wcs.pixelToSky(100.0, 100.0)

Let's try accessing the metadata, and see what (header) keywords we have.

In [ ]:
metadata = calexp.getMetadata()
pprint(metadata.toDict())

In [ ]:
#metadata.get('CCDTEMP')
metadata.get('TELESCOP')

## Better metadata: ExposureInfo and VisitInfo

For many purposes, information about an exposure is obtainable via the ExposureInfo and VisitInfo classes.

In [ ]:
calexp_info = calexp.getInfo()

In [ ]:
visit_info = calexp_info.getVisitInfo()

In [ ]:
[m for m in dir(visit_info) if not m.startswith('_')]

Obtain weather information for this visit

In [ ]:
visit_info.getWeather()

Check if this calexp has a valid polygon.

In [ ]:
calexp_info.hasValidPolygon()

There is no valid polygon for the `calexp`; so the `polygon` variable in the next cell returns `None`.

In [ ]:
polygon = calexp_info.getValidPolygon()
print(polygon)

The calexp is not a coadd so this method returns False.

In [ ]:
calexp_info.hasCoaddInputs()

Does the calexp contain transmission curve information?

In [ ]:
calexp_info.hasTransmissionCurve()

Does the calexp contain a World Coordinate System?

In [ ]:
calexp_info.hasWcs()

Does the calexp have a detector?

In [ ]:
calexp_info.hasDetector()

In [ ]:
[m for m in dir(calexp_info.getDetector()) if not m.startswith('_')]

Does the calexp have an aperture correction map?

In [ ]:
calexp_info.hasApCorrMap()

Let's get the aperture correction map and print some information about it

In [ ]:
apCorrMap = calexp_info.getApCorrMap()

In [ ]:
for k in apCorrMap.keys():
    print(k, apCorrMap.get(k))

## Image PSF

Check if our calexp has a PSF

In [ ]:
calexp.hasPsf()

In [ ]:
psf = calexp.getPsf()

The PSF object can be used to get a realization of a PSF at a specific point

In [ ]:
from lsst.geom import PointD
psfimage = psf.computeImage(PointD(100.,100.))

Visualize the PSF

In [ ]:
display = afwDisplay.Display()
display.scale('asinh', min=0.0, max=1.e-3, unit='absolute')
display.mtv(psfimage)

Access the calibration object which can be used to convert instrumental magnitudes to AB magnitudes

In [ ]:
calib = calexp.getPhotoCalib()
calib

## Image cutouts

We can make a cutout from the calexp in our session.

In [ ]:
import lsst.geom      as afwGeom
import lsst.afw.image as afwImage

In [ ]:
bbox = afwGeom.Box2I()
bbox.include(afwGeom.Point2I(400,1400))
bbox.include(afwGeom.Point2I(600,1600))

cutout = calexp[bbox]

Notice that when the image is displayed, the pixel values relate to the parent image.

In [ ]:
display = afwDisplay.Display()
display.scale('asinh', 'zscale')
display.mtv(cutout.image)

The coordinate of the lower-left-hand pixel is XY0.

In [ ]:
cutout.getXY0()

If a cutout was all that was desired from the start, we could have used our BoundingBox together with our Butler to have read in only the cutout.

In [ ]:
cutout_calexp = butler.get('calexp', parameters={'bbox':bbox}, dataId=dataId)
cutout_calexp.getDimensions()

In [ ]:
display = afwDisplay.Display()
display.scale('asinh', 'zscale')
display.mtv(cutout_calexp.image)

The `clone` method makes a deep copy. The result can be sliced with a BoundingBox

In [ ]:
clone_cutout = calexp.clone()[bbox]

In [ ]:
display = afwDisplay.Display()
display.scale("asinh", "zscale")
display.mtv(clone_cutout.image)

## Repeat for a coadd

For this last section, we will look at a coadd image. Earlier, we used the Butler to retrieve both calexp and coadd images.  Let's see here what methods it provides:

In [ ]:
coadd_methods = [m for m in dir(coadd) if not m.startswith('_')]

In [ ]:
coadd_methods

In [ ]:
set(coadd_methods).symmetric_difference(set(calexp_methods))

The result of the `set` command above shows that a calexp and a coadd have the same methods. This is expected, because they are instances of the same class (`ExposureF`).

In [ ]:
print(f"calexp class: {calexp.__class__}")
print(f"coadd  class: {coadd.__class__}")

A `deepCoadd` and a visit `calexp` differ mainly in the masks, xy0 values, and depth.

In [ ]:
print(f"calexp XY0: {calexp.getXY0()}")
calexp.mask.getMaskPlaneDict()

In [ ]:
print(f"coadd XY0: {coadd.getXY0()}")
coadd.mask.getMaskPlaneDict()

Display the coadd image

In [ ]:
display = afwDisplay.Display()
display.scale("asinh", "zscale")
display.mtv(coadd.image)